Preparing to submit wold stranded samples....

This got a little messy because a couple libraries needed a top up.

In [1]:
import os
import sys
import requests
import pandas
import paramiko
import json
from IPython import display
from pathlib import Path
import configparser

In [2]:
from curation_common import *
from htsworkflow.submission.encoded import DCCValidator

In [3]:
from htsworkflow.submission.encoded import Document
from htsworkflow.submission.aws_submission import run_aws_cp
from htsworkflow.util.api import (
    add_auth_options,
    make_auth_from_opts,
    HtswApi,
)

In [20]:
config = configparser.ConfigParser()
config.read([os.path.expanduser('~/.htsworkflow.ini'),
             '/etc/htsworkflow.ini'
             ])

SECTION = 'sequence_archive'
if config.has_section(SECTION):
    apiid = config.get(SECTION, 'apiid')
    apikey = config.get(SECTION, 'apikey')
    apihost = config.get(SECTION, 'host')

apihost='http://jumpgate.caltech.edu'    
auth = {'apiid': apiid, 'apikey': apikey }
htsw = HtswApi(apihost, auth)

In [5]:
# live server & control file
server = ENCODED('www.encodeproject.org')
spreadsheet_name = Path('~/woldlab/ENCODE/stranded-22588-22623-jax.xlsx').expanduser()
engine=None
#engine='odf'

# test server & datafile
#server = ENCODED('test.encodedcc.org')
#spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/C1-encode3-limb-2017-testserver.ods')

server.load_netrc()
validator = DCCValidator(server)

In [6]:
award = 'UM1HG009443'

# Lookup biosample ontologies

Lookup any biosample ontologies that are already present

In [7]:
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)

for i, row in biosample.iterrows():
    if not pandas.isnull(row.accession) and row.accession.startswith('E'):
        obj = server.get_json(row.accession)
        biosample_ontology = obj['biosample_ontology']
        if isinstance(biosample_ontology, dict):
            biosample.loc[i, 'biosample_ontology'] = biosample_ontology['@id']
            biosample.loc[i, 'biosample_term_name:skip'] = biosample_ontology['term_name']
        biosample.loc[i, 'source'] = obj['source']['@id']
            
biosample

,uuid,accession,aliases:array,biosample_ontology,biosample_term_name:skip,pooled_from:array,description,library_id:skip,cDNA_sample:skip,organism,source,lab,award
0,NaN,ENCBS928SLW,NaN,/biosample-types/tissue_UBERON_0001987/,placenta,NaN,Stam Placenta 11005,NaN,NaN,/organisms/human/,/sources/john-stamatoyannopoulos/,barbara-wold,UM1HG009443
1,NaN,ENCBS984FMU,NaN,/biosample-types/primary_cell_CL_0000624/,"CD4-positive, alpha-beta T cell",NaN,CD4+ T cells (CD3+CD4+),22588.0,ENC4_cDNA283,/organisms/human/,/sources/allcells/,barbara-wold,UM1HG009443
2,NaN,ENCBS075SKQ,NaN,/biosample-types/primary_cell_CL_0000624/,"CD4-positive, alpha-beta T cell",NaN,CD4+ T cells (CD3+CD4+),22589.0,ENC4_cDNA284,/organisms/human/,/sources/allcells/,barbara-wold,UM1HG009443
3,NaN,ENCBS883EPS,NaN,/biosample-types/primary_cell_NTR_0000495/,"activated CD4-positive, alpha-beta T cell",NaN,"CD4+ T cells, activated day 2",22590.0,ENC4_cDNA285,/organisms/human/,/sources/allcells/,barbara-wold,UM1HG009443
4,NaN,ENCBS555KPW,NaN,/biosample-types/primary_cell_NTR_0000495/,"activated CD4-positive, alpha-beta T cell",NaN,"CD4+ T cells, activated day 2",22591.0,ENC4_cDNA286,/organisms/human/,/sources/allcells/,barbara-wold,UM1HG009443
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

# Register Biosamples

In [18]:
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)
created = server.post_sheet('/biosamples/', biosample, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

0


In [ ]:
if created:
    biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

# Register Libraries

In [11]:
print(spreadsheet_name)
libraries = pandas.read_excel(spreadsheet_name, sheet_name='Library', header=0, engine=engine)
created = server.post_sheet('/libraries/', 
                            libraries,
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

/home/diane/woldlab/ENCODE/stranded-22588-22623-jax.xlsx
0


In [12]:
if created:
    libraries.to_excel('/dev/shm/libraries.xlsx', index=False)

In [29]:
for i, row in libraries.iterrows():
    accession = row['accession']
    alias = row['aliases:array']
    fragment = row['average_fragment_size:integer']
    library_id = alias[len('barbara-wold:'):]
    jumpgate_info = htsw.get_library(library_id)
    dcc_info = server.get_json(accession)
    print(library_id, fragment, jumpgate_info['insert_size'], dcc_info.get('average_fragment_size'))
    if jumpgate_info['insert_size'] != dcc_info.get('average_fragment_size'):
        server.patch_json(dcc_info['@id'], {'average_fragment_size': int(jumpgate_info['insert_size'])})

22588 267.0 161 267
22589 271.0 167 271
22590 259.0 171 259
22591 256.0 159 256
22592 278.0 156 278
22593 273.0 178 273
22594 285.0 173 285
22595 274.0 185 274
22596 285.0 174 285
22597 279.0 185 279
22598 256.0 179 256
22599 263.0 156 263
22600 264.0 163 264
22601 264.0 164 264
22602 267.0 164 267
22603 268.0 167 268
22604 267.0 168 267
22605 273.0 167 273
22606 272.0 173 272
22607 284.0 172 284
22608 300.0 184 300
22609 354.0 200 354
22610 368.0 254 368
22611 376.0 268 376
22612 367.0 276 367
22613 372.0 267 372
22614 368.0 272 368
22615 372.0 268 372
22616 370.0 272 370
22617 372.0 270 372
22618 371.0 272 371
22619 369.0 271 369
22620 364.0 269 364
22621 362.0 264 362
22622 371.0 262 371
22623 nan 271 None


# Generate library files for downloads

In [21]:
library_files = {}
for i, row in libraries.iterrows():
    library_id = row['aliases:array'].split(':')[1]
    jumpgate = htsw.get_library(library_id)
    for lane in jumpgate['lane_set']:
        #print(library_id, lane['flowcell'])
        if jumpgate['library_species'] == 'Homo sapiens':
            genome = 'GRCh38'
            annotation = 'V29'
            sex = 'male'
        else:
            raise RuntimeError('Unsupported species {}'.format(library_species))
        library_files.setdefault(lane['flowcell'], []).append({
            'library_id': library_id,
            'analysis_dir': library_id,
            'genome': genome,
            'annotation': annotation,
            'sex': sex,
            'read_1': Path(library_id) / (library_id + '*' + '.fastq.gz')
        })
    

In [ ]:
library_files.keys()

In [ ]:
for k in library_files:
    print(k, len(library_files[k]))

In [ ]:
pandas.DataFrame(library_files['HGLTKBCX3']).set_index('library_id')

In [ ]:
client = paramiko.SSHClient()
client.load_system_host_keys()
client.connect('pongo.caltech.edu')
sftp = client.open_sftp()

In [ ]:
flowcells = Path('/woldlab/loxcyc/home/diane/proj/flowcells')

In [ ]:
for key in library_files:
    curdir = flowcells / key
    try:
        stat = sftp.stat(str(curdir))
    except FileNotFoundError:
        print('{key} is missing'.format(key=key))
        sftp.mkdir(str(curdir))
        
    tsv = curdir / 'libraries.tsv'
    try:
        stat = sftp.stat(str(tsv))
    except FileNotFoundError:
        print('{tsv} is missing. Creating'.format(tsv=tsv))
        with sftp.open(str(tsv), 'w') as outstream:
            df = pandas.DataFrame(library_files[key]).set_index('library_id')
            df.to_csv(outstream, sep='\t')

# Register Experiments

In [14]:
print(server.server)
experiments = pandas.read_excel(spreadsheet_name, sheet_name='Experiment', header=0, engine=engine)
experiments = experiments[experiments['accession'] != 'barbara approval needed']
created = server.post_sheet('/experiments/', 
                            experiments, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

www.encodeproject.org
0


In [15]:
if created:
    experiments.to_excel('/dev/shm/experiments.xlsx', index=False)

# Register Replicates

In [22]:
print(server.server)
print(spreadsheet_name)
replicates = pandas.read_excel(spreadsheet_name, sheet_name='Replicate', header=0, engine=engine)
replicates = replicates[replicates['uuid'] != 'barbara approval needed']
created = server.post_sheet('/replicates/',
                            replicates, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

www.encodeproject.org
/home/diane/woldlab/ENCODE/stranded-22588-22623-jax.xlsx
0


In [21]:
if created:
    replicates.to_excel('/dev/shm/replicates.xlsx', index=False)

# Check Files

In [ ]:
files = pandas.read_excel(spreadsheet_name, sheet_name='File', header=0, engine=engine)
created = server.post_sheet('/files/', files, verbose=True, dry_run=True, validator=validator)
print(len(created))